In [1]:
import random
from lang import load_data_int_seq
from models.rl.env import IntegerSequenceEnv
from models.rl.agents.agent_mcts.main import train_mcts

%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 999.0

In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()
train = train[:1]

In [3]:
env = IntegerSequenceEnv({"data": train, "output_length": 9, "input_lang": input_lang, "output_lang": output_lang})

self.state
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [4]:
env.action_space.n

14

In [5]:
env.observation_space.shape[0]

18

In [7]:
train_mcts(env)


Training Episodes: 0
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Step: 0
Return: 0

Training Episodes: 0
[7, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Step: 1
Return: -999.0
ob
[-1 -1 -1 -1 -1 -1 -1 -1 -1  2  3  4  5  6  7  8  9 10]
ob
[ 7 -1 -1 -1 -1 -1 -1 -1 -1  2  3  4  5  6  7  8  9 10]
0
[-1 -1 -1 -1 -1 -1 -1 -1 -1  2  3  4  5  6  7  8  9 10  7 -1 -1 -1 -1 -1
 -1 -1 -1  2  3  4  5  6  7  8  9 10]


AssertionError: 